# Explosion Explorer

**By Caleb Buffa and Zach Phillips**

**Explosion events recorded in databases are usually confirmed by humans. But what happens if nobody is around, or survives, to confirm the event? Is it possible to estimate the probability that a suspected explosion occurred?**

**The Explosion Explorer Application gives users the ability to explore unpopulated places and examine the effects that explosions have on the surface of the Earth. Explosion Explorer leverages explosion events from the open-source Armed Conflict Location Event Database (ACLED) and before/after changes in indices derived from publicly available Landsat and Sentinel-2 products (NDVI, EVI, NBRT, and SAR) as training data for estimating the probability that an explosion occurred in a certain area, during a certain time period. Explosions included in the training dataset include shellings/artillery/missiles, air/drone strikes, landmines, remote triggered devices, suicide bombings, and grenades.**

**Red points initially displayed on the map are the ACLED points for explosions. Explore a known explosion location, or select a location and date for an unconfirmed location to estimate the probability that an explosion occurred in the selected location near the selected date.**

**Steps to estimate unconfirmed explosions:**

1. Activate the Map by clicking the box above the `Submit` button.
2. Use the map tools to drop a placemarker on the map
3. Select a Date to Investigate
4. Click the `Submit` button to estimate whether/not your point has been subjected to any explosions on the date selected.


In [1]:
import geemap
import ee
import ipywidgets as widgets
import ipyleaflet
import pickle
from IPython.display import display

from helper import calculate_difference

ee.Initialize()


In [ ]:
with open("explosion_model.sav", "rb") as in_model:
    clf = pickle.load(in_model)

In [4]:
# acled = pd.read_csv(r'2018-02-13-2021-02-17.csv')
# acled_sub = acled.loc[(acled['sub_event_type'] == 'Shelling/artillery/missile attack')]
# cols = ['longitude','latitude']
# coordinates = acled_sub[cols].values.tolist()

In [2]:
# Create an interactive map
Map = geemap.Map(center=[0, 0], zoom=2, add_google_map=False)
Map.add_basemap("HYBRID")

sentinel = ee.ImageCollection("COPERNICUS/S1_GRD_FLOAT")
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

# Adding date widget to map
date_picker = widgets.DatePicker(description="Pick a Date", disabled=False)
date_widget = widgets.Output(layout={"border": "1px solid black"})
date_control = ipyleaflet.WidgetControl(
    widget=date_widget, position="bottomright"
)
Map.add_control(date_control)

with date_widget:
    display(date_picker)

prediction_widget = widgets.Output(layout={"border": "1px solid black"})
prediction_control = ipyleaflet.WidgetControl(
    widget=prediction_widget, position="topright"
)
Map.add_control(prediction_control)

style = {"description_width": "initial"}

submit = widgets.Button(
    description="Submit",
    button_style="primary",
    tooltip="Click me",
    style=style,
)

aoi_widget = widgets.Checkbox(
    value=False,
    description="Drop point, pick date, and click Submit",
    style=style,
)
full_widget = widgets.VBox([widgets.HBox([aoi_widget]), submit])

full_control = ipyleaflet.WidgetControl(
    widget=full_widget, position="bottomright"
)
Map.add_control(full_control)


def submit_clicked(b):
    with prediction_widget:
        prediction_widget.clear_output()
        print("Computing...")
        try:
            roi = ee.FeatureCollection(Map.draw_last_feature)
            date = str(date_picker.value)
            if roi:
                if Map.draw_last_feature is not None:
                    roi = ee.FeatureCollection(Map.draw_last_feature)
                    if date_picker.value is not None:
                        calculate_difference(roi, date, Map, clf)
                    else:
                        pass
                else:
                    pass
            else:
                pass
        except Exception as e:
            print(e)
            print("An Error Occured During Computation")


submit.on_click(submit_clicked)

# cords = []
# for c in coordinates:
#     point = ee.Geometry.Point(c[0], c[1])
#     cords.append(point)
# fc = ee.FeatureCollection(cords)
# Map.addLayer(fc, {'color':'Red'}, 'ACLED Explosions')

Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Toggl…